In [2]:
import torch
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader, Dataset, random_split

# Summary-like from Tensorflow
from torchsummary import summary

# Import Numpy
import numpy as np
# Plot Import
import matplotlib.pyplot as plt
# For timer
import time
# Import OS
import os
# import local libs
import ecg_tools_lite as et

from statistics import mean

# WFDB
import wfdb

if not os.path.exists('./mlp_img'):
    os.mkdir('./mlp_img')

In [3]:
ecg_modded_06, ecg_untouched_06 = et.get_ecg_with_split('118e06', samp_freq=1024, norm_type='')
ecg_modded_24, ecg_untouched_24 = et.get_ecg_with_split('118e24', samp_freq=1024, norm_type='')

In [9]:
sample = torch.from_numpy(ecg_modded_06)

In [18]:
mean = ecg_modded_06.mean()
std = ecg_modded_06.std()
min_val = ecg_modded_06.min()
max_val = ecg_modded_06.max()
print( mean )
print( std )
print( min_val )
print( max_val )



-6.653124534970238
1.0670778518330393
-10.925
-2.07


Go read this article: 
https://stats.stackexchange.com/questions/178626/how-to-normalize-data-between-1-and-1